In [1]:
# %load app.py
"""
@author: 曾小青<zengxq@csust.edu.cn>
"""
import csv
import random

import numpy as np
import pandas as pd
from flask import Flask, render_template, request

import Recognizer as recog
import os

data\wordlib 字体图片文件夹存在，可以使用...
模型成功加载！


In [ ]:
app = Flask(__name__, static_folder="data", static_url_path="/static")

# 文件上传目录
app.config["UPLOAD_FOLDER"] = "data/uploads/"
app.config["TEST_FOLDER"] = "data/test/"
# 支持的文件格式
app.config["ALLOWED_EXTENSIONS"] = {
    "PNG",
    "JPG",
    "JPEG",
    "GIF",
    "png",
    "jpg",
    "jpeg",
    "gif",
}  # 集合类型


# 判断文件名是否是我们支持的格式
def allowed_file(filename):
    return (
        "." in filename
        and filename.rsplit(".", 1)[1] in app.config["ALLOWED_EXTENSIONS"]
    )


df_word_pinyin = pd.read_csv("data/word_pinyin.csv", header=0, encoding="gbk")
dict_word_pinyin = (
    df_word_pinyin[["word", "pinyin"]]
    .set_index("word")
    .to_dict(orient="dict")["pinyin"]
)


@app.route("/")
def home():
    return render_template("index.html")


@app.route("/input")
def new_input():
    return render_template("input.html")


@app.route("/show", methods=["POST"])
def show():
    upload_file = request.files["image_name"]
    if upload_file and allowed_file(upload_file.filename):
        filename = upload_file.filename
        # 将文件保存到 static/uploads 目录，文件名同上传时使用的文件名
        full_img_path = os.path.join(
            app.root_path, app.config["UPLOAD_FOLDER"], filename
        )
        upload_file.save(full_img_path)
        # 使用预见训练的TinyVGG模型进行预测
        # word, label = recog.predict_by_image_name(full_img_path, recog.loaded_model_0)
        # word_writer, label_writer = recog.predict_writer_by_image_name(full_img_path, recog.loaded_writer_model_0)

        # 使用预测训练的ResNet模型进行预测
        word,label=recog.predict_by_image_name(full_img_path,recog.loaded_model_0_resnet)
        word_writer,label_writer=recog.predict_writer_by_image_name(full_img_path,recog.loaded_writer_model_0_resnet)

        return render_template(
            "result.html",
            filename=filename,
            word=word,
            writer=word_writer,
            pinyin=dict_word_pinyin[word],
        )
    else:
        return "图像文件：" + filename + "上传失败，请检查文件否为(jpg,gif,png)类型..."


@app.route("/word_list", methods=["POST"])
def word_list():
    word = request.form["word"]
    images_name_list = recog.get_images_path_by_word(word)

    return render_template(
        "word_list.html",
        word=word,
        pinyin=dict_word_pinyin[word],
        name_list=images_name_list,
        base_url="/static/wordlib/",
    )


@app.route("/writer_list", methods=["POST"])
def writer_list():
    writer = request.form["writer"]
    images_name_list = recog.get_images_path_by_writer(writer)

    return render_template(
        "writer_list.html",
        writer=writer,
        name_list=images_name_list,
        base_url="/static/wordlib/",
    )


if __name__ == "__main__":
    app.secret_key = os.urandom(12)
    app.run(host="0.0.0.0", port=5556, debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5556
 * Running on http://10.171.132.48:5556
Press CTRL+C to quit
10.171.132.48 - - [15/Jan/2024 11:28:40] "GET / HTTP/1.1" 200 -
10.171.132.48 - - [15/Jan/2024 11:28:40] "GET /style.css HTTP/1.1" 404 -
10.171.132.48 - - [15/Jan/2024 11:28:40] "GET /jquery_1.12.0.min.js HTTP/1.1" 404 -
10.171.132.48 - - [15/Jan/2024 11:28:45] "POST /show HTTP/1.1" 200 -
10.171.132.48 - - [15/Jan/2024 11:28:45] "GET /static/uploads/190512.gif HTTP/1.1" 200 -


C:\Users\Administrator\Desktop\AI\CalligraphyRecognizer\data/uploads/190512.gif

图片文字预测为:"青"，其Label为8164
C:\Users\Administrator\Desktop\AI\CalligraphyRecognizer\data/uploads/190512.gif

图片文字预测为:"敬世江"，其Label为567


10.171.132.48 - - [15/Jan/2024 11:28:50] "GET /style.css HTTP/1.1" 404 -
10.171.132.48 - - [15/Jan/2024 11:28:59] "POST /show HTTP/1.1" 200 -
10.171.132.48 - - [15/Jan/2024 11:28:59] "GET /static/uploads/行书_黄庭坚_12941_873444_aug.gif HTTP/1.1" 200 -


C:\Users\Administrator\Desktop\AI\CalligraphyRecognizer\data/uploads/行书_黄庭坚_12941_873444_aug.gif

图片文字预测为:"傳"，其Label为337
C:\Users\Administrator\Desktop\AI\CalligraphyRecognizer\data/uploads/行书_黄庭坚_12941_873444_aug.gif

图片文字预测为:"黄庭坚"，其Label为1604


10.171.132.48 - - [15/Jan/2024 11:29:01] "GET /input HTTP/1.1" 200 -
10.171.132.48 - - [15/Jan/2024 11:29:01] "GET /jquery_1.12.0.min.js HTTP/1.1" 404 -
10.171.132.48 - - [15/Jan/2024 11:29:01] "GET /static/style.css HTTP/1.1" 200 -
10.171.132.48 - - [15/Jan/2024 11:29:01] "GET /static/images/search.gif HTTP/1.1" 200 -
10.171.132.48 - - [15/Jan/2024 11:29:06] "POST /word_list HTTP/1.1" 200 -
10.171.132.48 - - [15/Jan/2024 11:29:06] "GET /static/wordlib/曾_楷书_元倪墓志_113724.gif HTTP/1.1" 304 -
10.171.132.48 - - [15/Jan/2024 11:29:06] "GET /static/wordlib/曾_楷书_欧阳通_113719.gif HTTP/1.1" 304 -
10.171.132.48 - - [15/Jan/2024 11:29:06] "GET /static/wordlib/曾_楷书_褚遂良_113718.gif HTTP/1.1" 304 -
10.171.132.48 - - [15/Jan/2024 11:29:06] "GET /static/wordlib/曾_楷书_颜真卿_113720.gif HTTP/1.1" 304 -
10.171.132.48 - - [15/Jan/2024 11:29:06] "GET /static/wordlib/曾_楷书_颜真卿_113721.gif HTTP/1.1" 304 -
10.171.132.48 - - [15/Jan/2024 11:29:06] "GET /static/wordlib/曾_楷书_颜真卿_113722.gif HTTP/1.1" 304 -
10.171.132.48 - 